## Recurrent Neural Networks

In this chapter, we
are going to discuss recurrent neural networks (RNN), a class of nets that can predict
the future (well, up to a point, of course). They can analyze time series data such as
stock prices, and tell you when to buy or sell. In autonomous driving systems, they
can anticipate car trajectories and help avoid accidents. More generally, they can work
on sequences of arbitrary lengths, rather than on fixed-sized inputs like all the nets we
have discussed so far. For example, they can take sentences, documents, or audio
samples as input, making them extremely useful for natural language processing
(NLP) systems such as automatic translation, speech-to-text, or sentiment analysis
(e.g., reading movie reviews and extracting the rater’s feeling about the movie).
Moreover, RNNs’ ability to anticipate also makes them capable of surprising creativity.
You can ask them to predict which are the most likely next notes in a melody, then
randomly pick one of these notes and play it. Then ask the net for the next most likely
notes, play it, and repeat the process again and again. Before you know it, your net
will compose a melody such as the one produced by Google’s Magenta project. Similarly,
RNNs can generate sentences, image captions, and much more. The result is not
exactly Shakespeare or Mozart yet, but who knows what they will produce a few years
from now?
In this chapter, we will look at the fundamental concepts underlying RNNs, the main
problem they face (namely, vanishing/exploding gradients, discussed in Chapter 11),
and the solutions widely used to fight it: LSTM and GRU cells. Along the way, as
always, we will show how to implement RNNs using TensorFlow. Finally, we will take
a look at the architecture of a machine translation system.

## Recurrent Neurons